In [1]:
#Import Relevant Libraries
import random
import numpy as np
import cvxpy as cp

from qiskit import QuantumCircuit, execute, Aer, IBMQ, BasicAer
from qiskit.visualization import plot_histogram, plot_state_qsphere as plot_q
from qiskit.quantum_info import Statevector
from itertools import product #this one I use to find the prob of 00 without sampling. 

from toqito.state_metrics import fidelity
from toqito.channels import partial_trace as pt
from qiskit.quantum_info import DensityMatrix, state_fidelity, partial_trace

from qiskit.quantum_info import diamond_norm

from qiskit import quantum_info as qi
from toqito.channel_metrics import channel_fidelity

In [2]:
def rand_numb(low_in,high_in):
    '''
    Returns a random number between low_in and high_in, including both end points.
    '''
    return np.random.uniform(low=low_in, high=np.nextafter(high_in, np.inf))

In [3]:
'''
Defines parameters that specify the unitary extension of the channels.
'''
unitary_param_1 = [3.9979244, 0.0317666, 4.15727155, 3.33017616]
unitary_param_2 = [3.573949, 1.94640399, 3.36671537, 5.65794325]

In [4]:
'''
Create unitary extension of Channel 1
'''
size = 2
qc1 = QuantumCircuit(size)
for i in range(size):
    qc1.rx(unitary_param_1[i], i)
    qc1.ry(unitary_param_1[i+size], i)

for i in range(size-1):
    qc1.cx(i, i+1)

qc1.draw()

┌────────────┐ ┌────────────┐     
q_0: ─┤ Rx(3.9979) ├─┤ Ry(4.1573) ├──■──
     ┌┴────────────┴┐├────────────┤┌─┴─┐
q_1: ┤ Rx(0.031767) ├┤ Ry(3.3302) ├┤ X ├
     └──────────────┘└────────────┘└───┘

In [5]:
'''
Create unitary extension of Channel 2
'''
size = 2
qc2 = QuantumCircuit(size)
for i in range(size):
    qc2.rx(unitary_param_2[i], i)
    qc2.ry(unitary_param_2[i+size], i)

for i in range(size-1):
    qc2.cx(i, i+1)

qc2.draw()

┌────────────┐┌────────────┐     
q_0: ┤ Rx(3.5739) ├┤ Ry(3.3667) ├──■──
     ├────────────┤├────────────┤┌─┴─┐
q_1: ┤ Rx(1.9464) ├┤ Ry(5.6579) ├┤ X ├
     └────────────┘└────────────┘└───┘

In [6]:
'''
Creates extension of the Choi State of channel 1
'''
qua = QuantumCircuit(3)
qua.h(0)
qua.cx(0, 1)
qua.append(qc1, [1, 2])
qua.draw()

┌───┐                   
q_0: ┤ H ├──■────────────────
     └───┘┌─┴─┐┌────────────┐
q_1: ─────┤ X ├┤0           ├
          └───┘│  circuit-0 │
q_2: ──────────┤1           ├
               └────────────┘

In [7]:
'''
Creates extension of the Choi State of channel 2
'''
qua2 = QuantumCircuit(3)
qua2.h(0)
qua2.cx(0, 1)    
qua2.append(qc2, [1, 2])
qua2.draw()

┌───┐                   
q_0: ┤ H ├──■────────────────
     └───┘┌─┴─┐┌────────────┐
q_1: ─────┤ X ├┤0           ├
          └───┘│  circuit-1 │
q_2: ──────────┤1           ├
               └────────────┘

In [8]:
'''
Creates Choi matrices of the two channels
'''
psi_1 = np.array(DensityMatrix.from_instruction(qua.reverse_bits()))
psi_2 = np.array(DensityMatrix.from_instruction(qua2.reverse_bits()))
rho_1 = pt(psi_1, [3], [2, 2, 2])
rho_2 = pt(psi_2, [3], [2, 2, 2])

In [9]:
'''
Multiply the choi matrix by the normalization
'''
choi_1 = 2*rho_1
choi_2 = 2*rho_2

In [10]:
eye = [[1,0],[0,1]]
omega = cp.Variable((4,4),complex = True)
rho = cp.Variable((2,2),complex = True)
constraints = [omega>>0, rho>>0, omega << cp.kron(eye,rho), cp.trace(rho) == 1]
prob = cp.Problem(cp.Maximize(cp.real(cp.trace(omega@(choi_2 - choi_1)))),constraints)
normDiaDist = prob.solve(verbose = False)

WARN: A->p (column pointers) not strictly increasing, column 16 empty
WARN: A->p (column pointers) not strictly increasing, column 21 empty
WARN: A->p (column pointers) not strictly increasing, column 26 empty
WARN: A->p (column pointers) not strictly increasing, column 31 empty


In [11]:
print("Normalized Diamond Distance : ", normDiaDist)

Normalized Diamond Distance :  0.45158228621024654
